# Exercises

Learning a language requires _using_ it. This is why I can no longer speak French. 

This page gives gives a series of exercises for basic analyses of data in the Datajoint pipeline. 

If you have not yet reconfigured your Datajoint client to access the latest version of the pipeline, you can find details [on the kavli wiki here](https://www.ntnu.no/wiki/display/kavli/DataJoint%3A+Electrophysiology+Pipeline). You will require your NTNU username, database password, and the access and secret keys to the object storage. 

In [ ]:
import os
import scipy.ndimage
import datetime
import numpy as np
import datajoint as dj
import matplotlib.pyplot as plt

SECRET_KEY = ""
ACCESS_KEY = ""
USER = ""
PASS = ""

dj.config['database.host'] = 'datajoint.it.ntnu.no'
dj.config['database.user'] = USER
dj.config['database.password'] = PASS
dj.config["enable_python_native_blobs"] = True
dj.config["stores"] = {   'ephys_store': {   'access_key': ACCESS_KEY,
                                     'bucket': 'ephys-store-computed',
                                     'endpoint': 's3.stack.it.ntnu.no:443',
                                     'secure': True,
                                     'location': '',
                                     'protocol': 's3',
                                     'secret_key': SECRET_KEY},
                  'ephys_store_manual': {   'access_key': ACCESS_KEY,
                                            'bucket': 'ephys-store-manual',
                                            'endpoint': 's3.stack.it.ntnu.no:443',
                                            'secure': True,
                                            'location': '',
                                            'protocol': 's3',
                                            'secret_key': SECRET_KEY}}
dj.config['custom'] = {
        'database.prefix': 'group_shared_',
        'mlims.database': 'prod_mlims_data',
        'flask.database': 'group_shared_flask',
        'drive_config': {
          'local': 'C:/',
          'network': 'N:/'}}

analysis = dj.create_virtual_module('analysis', 'group_shared_analysis')
analysis_param = dj.create_virtual_module('analysis_param', 'group_shared_analysis_param')

key = {'animal_id': 'f5e35afe2bf2a9f8',
 'datasource_id': 0,
 'session_time': datetime.datetime(2019, 11, 11, 12, 45, 59),
 'unit': 167,
 'task_type': 'OpenField',
 'task_start': '21.00',
 'task_spike_tracking_hash': '083ff30e220fbe26815e820e54451af6',
 'occu_params_name': 'default',
 'smoothing_params_name': 'default',
 'analysis_package': 'python',
 'cell_selection_params_name': 'default',
 'field_detect_params_name': 'default',
 'score_params_name': 'default'}

## Exercise 1: Path maps

In the first exercise, you will plot a path and spike map

Goal: given a `key` to the Datajoint table `analysis.TaskSpikesTracking`, create and display a path Map for that cell in that task. 

This one is fairly simple: due to the convenience of Datajoint, you need only fetch four arrays from the database, and plot them directly using `matplotlib`. Don't forget to set the correct aspect ratio. Further, because this is about using Python correctly, not Datajoint in particular, the below code will do the database hevay lifting for you

In [ ]:
pos_t, pos_x, pos_y = (analysis.TaskTracking & key).fetch1("task_timestamps", "x_pos", "y_pos")
spk_t, spk_x, spk_y = (analysis.TaskSpikesTracking & key).fetch1("spike_times", "x_pos", "y_pos")

## Exercise 2: Coverage

It's useful to know how much of the arena the animal has visited. This requires some manipulation of the data you have acquired so far. IWhat assumptions are you making?

**Hint**: You will find the [NumPy Documentation](https://docs.scipy.org/doc/numpy) very useful, and the [2D histogram](https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram2d.html) function in particular, and it's hard to get anything done with arrays without [meshgrid](https://docs.scipy.org/doc/numpy/reference/generated/numpy.meshgrid.html)

## Exercise 3: Rate maps

A ratemap takes in a list of animal positions, and a list of spike positions, and computes the rate of firing of that unit at each location in an arena. 

Based on these data, create:
* An occupancy map: a binned map of the time the animal spent in each location
* A spike map: a binned map of the number of unit spikes in each location
* A rate map: the rate at which the unit fires in each location





# Exercise 4: Smoothing

Coarse discretisation like this is highly prone to distortionary boundary effects, which can be ameliorated by smoothing. Conventionally, by smoothing, we mean that each pixel is made more like its neighbours. The fascinating detail (and a significant part of the engineering-physics-maths sub-field of signal processing) is how neighbours are weighted in this calculation

This is where analysis starts to become extremely complicated by the distressing existence of boundaries. 

How we we handle unexpected values?

The simplest solution for convolution in Python is to use [`scipy.ndimage.filters.convolve`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.ndimage.filters.convolve.html)

This requires a `weights` keyword that determines how we weight the value of each neighbour.